<a href="https://colab.research.google.com/github/orodriguezq/orodriguezq-Senales_Y_Sistemas/blob/main/Talleres/Youtube_Detector_Dashboard_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🔧 1) Instalar dependencias

In [185]:
!pip install -q streamlit yt-dlp pydub joblib scikit-learn matplotlib requests ffmpeg
!apt update && apt install -y ffmpeg
!ffmpeg -version


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
35 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

2) (Opcional) Limpia antiguos directorios

In [186]:
!rm -rf pages


3) 0_👋_Hello.py completo

In [187]:
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import joblib
import io
import os
import re
import traceback
from yt_dlp import YoutubeDL
from pydub import AudioSegment
from sklearn.preprocessing import MinMaxScaler

# --- Configuración de la app ---
st.set_page_config(page_title="Detección de Género Musical", page_icon="🎵")
st.markdown("# Detección de Género Musical 🎵")
st.sidebar.header("Enlace de YouTube o ID (11 caracteres)")

# --- Carga del modelo ---
MODEL_FILE = "knn_genre_detector.pkl"
if not os.path.exists(MODEL_FILE):
    st.error(f"❌ Modelo no encontrado: {MODEL_FILE}")
    st.stop()

@st.cache_data
def load_model():
    return joblib.load(MODEL_FILE)

loaded = load_model()
if isinstance(loaded, dict):
    # extraemos el primer estimador con predict
    model = next((v for v in loaded.values() if hasattr(v, 'predict')), None)
    if model is None:
        st.error("No se encontró un estimador en el pickle.")
        st.stop()
else:
    model = loaded

# --- Función para descargar audio completo ---
@st.cache_data
def download_full_audio(video_id: str) -> AudioSegment:
    # limpia restos previos
    for f in os.listdir():
        if f.startswith("yt_audio."):
            try: os.remove(f)
            except: pass
    opts = {'format': 'bestaudio/best', 'outtmpl': 'yt_audio.%(ext)s', 'quiet': True}
    url = f"https://www.youtube.com/watch?v={video_id}"
    with YoutubeDL(opts) as ydl:
        info = ydl.extract_info(url, download=True)
        ext = info.get('ext', 'm4a')
        fname = f"yt_audio.{ext}"
    audio = AudioSegment.from_file(fname)
    try: os.remove(fname)
    except: pass
    return audio

# --- Interfaz de usuario ---
raw = st.sidebar.text_input("Pega URL o ID de YouTube", "").strip()
if raw:
    m = re.search(r"([A-Za-z0-9_-]{11})", raw)
    if not m:
        st.error("❌ Formato no válido. Introduce URL o ID de 11 caracteres de YouTube.")
        st.stop()
    vid = m.group(1)
    try:
        with st.spinner("Descargando audio completo..."):
            audio_full = download_full_audio(vid)
        # segment de 15 segundos centrado
        total_s = int(audio_full.duration_seconds)
        seg_len = min(15, total_s)
        start_s = max((total_s - seg_len) // 2, 0)
        segment = audio_full[start_s*1000:(start_s+seg_len)*1000]

        # Reproducción del segmento
        buf = io.BytesIO()
        segment.export(buf, format="wav")
        buf.seek(0)
        st.subheader(f"Audio ({seg_len}s) desde {start_s}s")
        st.audio(buf.getvalue(), format="audio/wav")

        # Procesamiento numpy
        fs = segment.frame_rate
        samples = np.array(segment.get_array_of_samples())
        x = samples.reshape(-1, 2) if segment.channels == 2 else np.stack([samples, samples], axis=-1)

        # 1) Dominio del tiempo
        fig1, ax1 = plt.subplots()
        t = np.arange(x.shape[0]) / fs
        ax1.plot(t, x.mean(axis=1))
        ax1.set(xlabel="t [s]", ylabel="x(t)")
        st.pyplot(fig1)

        # 2) Dominio de la frecuencia
        vf = np.fft.rfftfreq(x.shape[0], 1/fs)
        Xw = np.fft.rfft(x, axis=0).mean(axis=-1)
        fig2, ax2 = plt.subplots()
        ax2.plot(vf, np.abs(Xw))
        ax2.set(xlabel="f [Hz]", ylabel="|X(f)|")
        st.pyplot(fig2)

        # 3) Normalización
        scaler = MinMaxScaler()
        Xn = scaler.fit_transform(np.abs(Xw).reshape(-1,1)).flatten()
        fig3, ax3 = plt.subplots()
        ax3.plot(vf, Xn)
        ax3.set(xlabel="f [Hz]", ylabel="|X(f)| normalizado")
        st.pyplot(fig3)

        # 4) Espectro en dB
        fig4, ax4 = plt.subplots()
        ax4.plot(vf, 20*np.log10(Xn + 1e-10))
        ax4.set(xlabel="f [Hz]", ylabel="|X(f)| dB")
        st.pyplot(fig4)

        # Ajustar features al modelo
        expected = getattr(model, 'n_features_in_', len(Xn))
        if len(Xn) > expected:
            Xn_adj = Xn[:expected]
        else:
            Xn_adj = np.pad(Xn, (0, expected-len(Xn)), 'constant')

        # 5) Predicción y mapeo de etiquetas
        raw_pred = model.predict(Xn_adj.reshape(1, -1))[0]
        label_map = {1: 'Reggaeton', 2: 'Salsa'}
        genero = label_map.get(raw_pred, f"Clase {raw_pred}")
        st.subheader("Predicción de Género")
        st.success(f"**Género previsto:** {genero}")

    except Exception as e:
        st.error(f"🚨 Error procesando: {e}")
        st.text(traceback.format_exc())
else:
    st.info("👉 Ingresa URL o ID en la barra lateral.")


2025-06-16 04:14:57.272 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 04:14:57.274 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 04:14:57.275 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 04:14:57.277 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 04:14:57.278 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 04:14:57.279 No runtime found, using MemoryCacheStorageManager
2025-06-16 04:14:57.285 No runtime found, using MemoryCacheStorageManager
2025-06-16 04:14:57.288 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-16 04:14:57.289 Thread 'MainThread': missing ScriptRunContext! This warning can be ignor

4) Arrancar Streamlit y Cloudflared

In [192]:
# Mata cualquier instancia previa
!pkill -f streamlit

# Muestra los últimos 30 líneas de tus logs
!tail -n 30 /content/logs.txt || echo "No hay logs.txt aún"


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: 0_👋_Hello.py


In [193]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

# Ejecutar Streamlit en background
!streamlit run 0_👋_Hello.py &> /content/logs.txt &

# Iniciar túnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &



In [194]:
import time, re, os

time.sleep(5)
url = None
with open('/content/cloudflared.log') as f:
    for line in f:
        if "Your quick Tunnel has been created" in line:
            m = re.search(r'https?://\S+', line)
            if m:
                url = m.group(0)
                break

if url:
    print(f"✅ Tu app está en:\n{url}")
else:
    print("⚠️ No encontré la URL. Revisa cloudflared.log:\n")
    print(open('/content/cloudflared.log').read())

res = input("Escribe '1' para detener Streamlit: ")
if res.strip() == "1":
    os.system("pkill streamlit")
    print("🔴 Streamlit detenido.")

⚠️ No encontré la URL. Revisa cloudflared.log:

2025-06-16T04:17:01Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-06-16T04:17:01Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-06-16T04:17:05Z INF +--------------------------------------------------------------------------------------------+
2025-06-16T04:17:05Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-06-16T04:17:05Z INF |  https